In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

In [1]:
import os
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

In [2]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                     TextMessage(text=event.message.text)]
            )
        )

@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2024 12:42:18] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}


127.0.0.1 - - [08/May/2024 13:52:07] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"507250775421091973","quoteToken":"ifDElG-vTZ_0tC_Zb2FL1PMcUpnOaUnxi88W5KLU0E2Fs6rv1VAcA9fFdDy1-k1d0DTf7IRLqPVul-PlK-4sfzALoaAbgoMfXPcUByR7zMBeiX96gK-8_GJw_yAm_d0Q9syE02CR-XWW9bjCDGfJ0w","text":"四"},"webhookEventId":"01HXC7KJ50SJMGQAX0BN79PBJ5","deliveryContext":{"isRedelivery":false},"timestamp":1715176326821,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"8f01ef5a1d3f4995a352a10b3bbf8884","mode":"active"}]}
